In [ ]:
import torch

import econpy as ep
from econpy.lin.tests import test_err, test_est

In [ ]:
n = 2**14
d = 2**10

def main():
    x = torch.empty((d, n), device=ep.lin.base.CUDA).normal_()
    fe1 = torch.randint(0, 2**10, (n,), device=ep.lin.base.CUDA)
    fe2 = torch.randint(0, 2**8, (n,), device=ep.lin.base.CUDA)
    fe3 = torch.randint(0, 2**4, (n,), device=ep.lin.base.CUDA)
    fes = torch.stack((fe1, fe2, fe3))
    fe_feats1 = torch.cat([torch.ones((1, n), device=ep.lin.base.CUDA)] + [
        torch.zeros((fe.max() + 1, n), device=ep.lin.base.CUDA).scatter_(
            0, fe.expand(fe.max()+1, n), 1.)[:-1]
        for fe in fes])
    fe1 = torch.randint(0, 2**8, (n,), device=ep.lin.base.CUDA)
    fe2 = torch.randint(0, 2**6, (n,), device=ep.lin.base.CUDA)
    fe3 = torch.randint(0, 2**5, (n,), device=ep.lin.base.CUDA)
    fe_trends = torch.stack((fe1, fe2, fe3))
    fe_trend_ws = torch.stack(tuple(
        torch.rand(n, device=ep.lin.base.CUDA) for _ in range(len(fe_trends))))
    fe_feats = torch.cat([
        torch.zeros((fe.max() + 1, n), device=ep.lin.base.CUDA).scatter_(
            0, fe.expand(fe.max() + 1, n), 1) * fe_w
        for fe, fe_w in zip(fe_trends, fe_trend_ws)])
    fe_feats = torch.cat((fe_feats1, fe_feats))
    y = ep.lin.pre.absorb_fes(
        x.clone(), fes, fe_trends, fe_trend_ws, eps=2**(-11))
    z = ep.lin.pre.absorb(x.clone(), fe_feats)
    err1 = torch.linalg.matrix_norm(z - y, 2).div_(
        torch.linalg.matrix_norm(x).div_(d**0.5))
    err2 = (z - y).abs().max() / x.abs().median()
    print(err1, err2)

In [ ]:
main()

In [ ]:
torch.rand((2**2, 2**8)).median(1).values

In [ ]:
test_est.test()

In [ ]:
test_err.test(ep.lin.err.const)

In [ ]:
r = torch.zeros((d, 10), device=ep.lin.base.CUDA)
r.scatter_add_(1, fe3.expand(d, -1), y)